# 3. Streaming application using Spark Structured Streaming<a class="anchor" id="1"></a>
## 3.1 Spark session<a class="anchor" id="1.1"></a>

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0 pyspark-shell'

from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import *
import datetime as dt
from pyspark.ml import PipelineModel

from pyspark import SparkConf # Spark
from pyspark.sql import SparkSession

spark_conf = SparkConf()\
            .setMaster("local[2]")\
            .setAppName("Assignment2B-Task3- Morad Abou Shadi").set("spark.sql.session.timeZone", "UTC")

spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
spark.sparkContext.setLogLevel('ERROR')

## 3.2 Streaming data ingestion<a class="anchor" id="1.1"></a>

In [2]:
topic = "flightTopic"
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", topic) \
    .load()

In [3]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



## 3.3 Data formatting<a class="anchor" id="1.1"></a>

In [4]:
schema = ArrayType(StructType([    
    StructField('YEAR', StringType(), True), 
    StructField('MONTH', StringType(), True),
    StructField('DAY', StringType(), True),
    StructField('DAY_OF_WEEK', StringType(), True),
    StructField('AIRLINE', StringType(), True),
    StructField('FLIGHT_NUMBER', StringType(), True),
    StructField('TAIL_NUMBER', StringType(), True),
    StructField('ORIGIN_AIRPORT', StringType(), True),
    StructField('DESTINATION_AIRPORT', StringType(), True),
    StructField('SCHEDULED_DEPARTURE', StringType(), True),
    StructField('DEPARTURE_TIME', StringType(), True),
    StructField('DEPARTURE_DELAY', StringType(), True),
    StructField('TAXI_OUT', StringType(), True),
    StructField('WHEELS_OFF', StringType(), True),
    StructField('SCHEDULED_TIME', StringType(), True),
    StructField('ELAPSED_TIME', StringType(), True),
    StructField('AIR_TIME', StringType(), True),
    StructField('DISTANCE', StringType(), True),
    StructField('WHEELS_ON', StringType(), True),
    StructField('TAXI_IN', StringType(), True),
    StructField('SCHEDULED_ARRIVAL', StringType(), True),
    StructField('ARRIVAL_TIME', StringType(), True),
    StructField('ARRIVAL_DELAY', StringType(), True),
    StructField('DIVERTED', StringType(), True),
    StructField('CANCELLED', StringType(), True),
    StructField('CANCELLATION_REASON', StringType(), True),
    StructField('AIR_SYSTEM_DELAY', StringType(), True),
    StructField('SECURITY_DELAY', StringType(), True),
    StructField('AIRLINE_DELAY', StringType(), True),
    StructField('LATE_AIRCRAFT_DELAY', StringType(), True),
    StructField('WEATHER_DELAY', StringType(), True),
    StructField('ts', StringType(), True)            
]))

In [5]:
df = df.select(F.from_json(F.col("value").cast("string"), schema).alias('value'))
df = df.select(F.explode(F.col("value")).alias('value'))     

In [6]:
df.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- YEAR: string (nullable = true)
 |    |-- MONTH: string (nullable = true)
 |    |-- DAY: string (nullable = true)
 |    |-- DAY_OF_WEEK: string (nullable = true)
 |    |-- AIRLINE: string (nullable = true)
 |    |-- FLIGHT_NUMBER: string (nullable = true)
 |    |-- TAIL_NUMBER: string (nullable = true)
 |    |-- ORIGIN_AIRPORT: string (nullable = true)
 |    |-- DESTINATION_AIRPORT: string (nullable = true)
 |    |-- SCHEDULED_DEPARTURE: string (nullable = true)
 |    |-- DEPARTURE_TIME: string (nullable = true)
 |    |-- DEPARTURE_DELAY: string (nullable = true)
 |    |-- TAXI_OUT: string (nullable = true)
 |    |-- WHEELS_OFF: string (nullable = true)
 |    |-- SCHEDULED_TIME: string (nullable = true)
 |    |-- ELAPSED_TIME: string (nullable = true)
 |    |-- AIR_TIME: string (nullable = true)
 |    |-- DISTANCE: string (nullable = true)
 |    |-- WHEELS_ON: string (nullable = true)
 |    |-- TAXI_IN: string (nullable = true)
 |    |

In [7]:
df = df.select(
                    F.col("value.YEAR").cast("int").alias("YEAR"),
                    F.col("value.MONTH").cast("int").alias("MONTH"),
                    F.col("value.DAY").cast("int").alias("DAY"),
                    F.col("value.DAY_OF_WEEK").cast("int").alias("DAY_OF_WEEK"),
                    F.col("value.AIRLINE").alias("AIRLINE"),
                    F.col("value.FLIGHT_NUMBER").alias("FLIGHT_NUMBER"),
                    F.col("value.TAIL_NUMBER").alias("TAIL_NUMBER"),
                    F.col("value.ORIGIN_AIRPORT").alias("ORIGIN_AIRPORT"),
                    F.col("value.DESTINATION_AIRPORT").alias("DESTINATION_AIRPORT"),
                    F.col("value.SCHEDULED_DEPARTURE").cast("int").alias("SCHEDULED_DEPARTURE"),
                    F.col("value.DEPARTURE_TIME").cast("int").alias("DEPARTURE_TIME"),
                    F.col("value.DEPARTURE_DELAY").cast("int").alias("DEPARTURE_DELAY"),
                    F.col("value.TAXI_OUT").cast("int").alias("TAXI_OUT"),
                    F.col("value.WHEELS_OFF").cast("int").alias("WHEELS_OFF"),
                    F.col("value.SCHEDULED_TIME").cast("int").alias("SCHEDULED_TIME"),
                    F.col("value.ELAPSED_TIME").cast("int").alias("ELAPSED_TIME"),
                    F.col("value.AIR_TIME").cast("int").alias("AIR_TIME"),
                    F.col("value.DISTANCE").cast("int").alias("DISTANCE"),
                    F.col("value.WHEELS_ON").cast("int").alias("WHEELS_ON"),
                    F.col("value.TAXI_IN").cast("int").alias("TAXI_IN"),
                    F.col("value.SCHEDULED_ARRIVAL").cast("int").alias("SCHEDULED_ARRIVAL"),
                    F.col("value.ARRIVAL_TIME").cast("int").alias("ARRIVAL_TIME"),
                    F.col("value.ARRIVAL_DELAY").cast("int").alias("ARRIVAL_DELAY"),
                    F.col("value.DIVERTED").cast("int").alias("DIVERTED"),
                    F.col("value.CANCELLED").cast("int").alias("CANCELLED"),
                    F.col("value.CANCELLATION_REASON").alias("CANCELLATION_REASON"),
                    F.col("value.AIR_SYSTEM_DELAY").cast("int").alias("AIR_SYSTEM_DELAY"),
                    F.col("value.SECURITY_DELAY").cast("int").alias("SECURITY_DELAY"),
                    F.col("value.AIRLINE_DELAY").cast("int").alias("AIRLINE_DELAY"),
                    F.col("value.LATE_AIRCRAFT_DELAY").cast("int").alias("LATE_AIRCRAFT_DELAY"),
                    F.col("value.WEATHER_DELAY").cast("int").alias("WEATHER_DELAY"),
                    F.col("value.ts").cast("int").alias("ts")
                )

In [8]:
df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: string (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (nulla

## 3.4 Output sink creation (console)<a class="anchor" id="1.1"></a>

In [9]:
query = df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime='5 seconds') \
    .start()

In [10]:
query.stop()

## 3.5 Dataframe clean up + binaryArrDelay column<a class="anchor" id="1.1"></a>

In [11]:
df = df.withColumn("binaryArrDelay",F.when((F.col("ARRIVAL_DELAY") > 0),1).otherwise(0))

In [12]:
rowsToDrop =  ['CANCELLATION_REASON','AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY','LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
df = df.drop(*rowsToDrop)

In [13]:
df = df.withColumn("DEPT_TIME_FLAG",F.when((F.col("SCHEDULED_DEPARTURE").between(0,459 )),"Midnight").when((F.col("SCHEDULED_DEPARTURE").between(500,1059)),"Morning").when((F.col("SCHEDULED_DEPARTURE").between(1100,1559)),"Afternoon").when((F.col("SCHEDULED_DEPARTURE").between(1600,1959)),"Evening").otherwise("Night"))

In [14]:
df = df.na.drop()

## 3.6 Machine learning model<a class="anchor" id="1.1"></a>

In [15]:
from zipfile import ZipFile
zt = ZipFile('binary_arrival_delay_pipeline_model.zip')
zt.extractall()

In [16]:
from pyspark.ml import PipelineModel

model = PipelineModel.load('binary_arrival_delay_pipeline_model')

In [17]:
predictions = model.transform(df)

In [18]:
agg_predictions = predictions.groupBy('prediction').count()

## 3.7 Output sink creation (Memory) + prediction results<a class="anchor" id="1.1"></a>

In [19]:
import pyspark.sql.functions as F

cnt_cond = lambda cond: F.sum(F.when(cond, 1).otherwise(0))

agg_predictions = predictions.groupBy(window(col("ts").cast("timestamp"), "20 seconds"),col("DAY_OF_WEEK"))\
.agg((cnt_cond(col('prediction') == col('binaryArrDelay'))/count(col('prediction'))).alias("Accuracy"), count(col('DAY_OF_WEEK')).alias('count')).sort(desc("window")).filter('DAY_OF_WEEK < 4')

In [20]:
prediction_query = agg_predictions \
    .writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("prediction") \
    .trigger(processingTime='20 seconds') \
    .start()

In [31]:
prediction_query.stop()

In [22]:
spark.sql("select * from prediction").show(truncate=False)

+------------------------------------------+-----------+------------------+-----+
|window                                    |DAY_OF_WEEK|Accuracy          |count|
+------------------------------------------+-----------+------------------+-----+
|[2022-05-26 23:29:40, 2022-05-26 23:30:00]|2          |0.2               |5    |
|[2022-05-26 23:29:40, 2022-05-26 23:30:00]|1          |0.3333333333333333|3    |
|[2022-05-26 23:29:40, 2022-05-26 23:30:00]|3          |0.25              |4    |
|[2022-05-26 23:29:20, 2022-05-26 23:29:40]|2          |0.0               |1    |
|[2022-05-26 23:29:20, 2022-05-26 23:29:40]|1          |0.0               |2    |
|[2022-05-26 23:29:20, 2022-05-26 23:29:40]|3          |0.3333333333333333|3    |
|[2022-05-26 23:29:00, 2022-05-26 23:29:20]|3          |1.0               |2    |
|[2022-05-26 23:29:00, 2022-05-26 23:29:20]|2          |0.5               |2    |
|[2022-05-26 23:29:00, 2022-05-26 23:29:20]|1          |0.0               |1    |
+---------------